# Classification of Plant Leaf Diseases using MLPs (32x32 images)

## List of experiments 
1. 3 layer MLP w/ homebrew network
2. 3 layer MLP w/ pytorch

In [46]:
# General imports 
import sys
import os 
sys.path.insert(1, os.path.join(os.pardir, 'src'))

# Data imports
import cv2
import plotly.express as px
import numpy as np
import mlflow

# Homebrew imports 
import model
from utils import one_hot_encode_index
from optimizers import Adam
from activations import Softmax, ReLU
from layers import Dropout, LinearLayer
from loss import CategoricalCrossEntropyLoss

## Loading & Preprocessing Data 

In [35]:
data_path = os.path.join(os.pardir, 'data', 'Plant_leave_diseases_32')

X = []
y = []

for _class in os.listdir(data_path)[:1]:
    for img_name in os.listdir(os.path.join(data_path, _class))[:100]:
        img = cv2.imread(os.path.join(data_path, _class, img_name), cv2.IMREAD_GRAYSCALE)
        X.append(img)
        y.append(_class)

X = np.array(X)

# Scaling X between -1, 1  
X = (X - np.mean(X))/ np.mean(X)

# Vectorizing X
X = X.reshape(X.shape[0], -1)

# encoding y
class_lbl_to_int = dict([(y, i) for i, y in enumerate(sorted(set(y)))])
y = np.array([class_lbl_to_int[_y] for _y in y]).astype(np.uint8)
y = one_hot_encode_index(y, len(class_lbl_to_int))


## Shuffling & splitting data

Since the data is large we will use a random 90,5,5 split for training, validation and testing respectively 

In [44]:
n_samples = X.shape[0]
keys = np.array(range(X.shape[0]))
np.random.shuffle(keys)

train_keys = keys[:int(n_samples*0.9)]
validation_keys = keys[int(n_samples*0.9):int(n_samples*0.9 + n_samples*0.05)]
test_keys = keys[int(n_samples*0.9 + n_samples*0.05):]

X_train = X[train_keys]
X_val = X[validation_keys]
X_test = X[test_keys]

y_train = y[train_keys]
y_val = y[validation_keys]
y_test = y[test_keys]

## 3 Layer MLP - Homebrew

In [45]:
# Set up optimizer and Loss objects 
cce_loss = CategoricalCrossEntropyLoss()
optimizer = Adam(decay=5e-5)

# initialize model 
my_model = model.Model(optimizer, cce_loss)

# Configuring model save path 
my_model.set_save_config(model_name='3_layer_mlp_HB', save_path=os.path.join(os.getcwd(), 'models'))

# Defining architecture 
my_model.set_sequence([
                LinearLayer(X.shape[1], 682),
                ReLU(),
                LinearLayer(682, 456),
                ReLU(),
                LinearLayer(456, len(class_lbl_to_int)),
                Softmax()
            ])

In [ ]:
mlflow.set_experiment(experiment_name='Plant Leaf Disease')

tags = {
    'model_type':'mlp-homebrew',
    'image_size': '32x32'
}